In [ ]:
%load_ext autoreload
%autoreload 2

In [3]:
import ipas.collection_no_db.iceice_collection as collect
import numpy as np
from dask_jobqueue import SLURMCluster
from dask.distributed import Client
import dask
import pandas as pd
import pickle

In [4]:
cluster = SLURMCluster(
queue='kratos',
walltime='04-23:00:00',
cores=1,
memory='20000MiB', #1 GiB = 1,024 MiB
processes=1)

cluster.scale(1)
client = Client(cluster)
print(client)

<Client: 'tcp://169.226.65.141:38313' processes=0 threads=0, memory=0 B>


/network/rit/lab/sulialab/share/bin/miniconda3/envs/IPAS_v1.3.0/lib/python3.7/site-packages/distributed/node.py:155: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 39033 instead
  http_address["port"], self.http_server.port


In [5]:
client

Client Scheduler: tcp://169.226.65.141:38313 Dashboard: http://169.226.65.141:39033/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [7]:
def main():
    output = np.empty((len(phioarr),len(reqarr)),dtype=object)
    for phi in range(len(phioarr)):
        for r in range(len(reqarr)):
            output[phi,r] = dask.delayed(collect.collect_clusters_ice_ice)(phioarr[phi], reqarr[r], nclusters, ncrystals,rand_orient)
            #collect.collect_clusters_ice_ice(phioarr[phi], reqarr[r], nclusters, ncrystals,rand_orient)

    return output

In [8]:
def compute():
    agg_as = np.empty((len(phioarr),len(reqarr), nclusters))
    agg_bs = np.empty((len(phioarr),len(reqarr), nclusters))
    agg_cs = np.empty((len(phioarr),len(reqarr), nclusters))
    phi2Ds = np.empty((len(phioarr),len(reqarr), nclusters))
    cplxs = np.empty((len(phioarr),len(reqarr), nclusters))
    dds = np.empty((len(phioarr),len(reqarr), nclusters))

    gather = client.compute([*output.tolist()])  #only parallelizing agg r bins
    gather = client.gather(gather)

    gather = np.array(gather)
    print(np.shape(gather))
    agg_as = gather[:,:,0,:]
    agg_bs = gather[:,:,1,:]
    agg_cs = gather[:,:,2,:]
    phi2Ds = gather[:,:,3,:]
    cplxs = gather[:,:,4,:] 
    dds = gather[:,:,5,:]

    print('DONE!')
    return agg_as, agg_bs, agg_cs, phi2Ds, cplxs, dds

In [10]:
if __name__ == '__main__':
    phioarr=np.logspace(-1.5, 2, num=20, dtype=None)#just columns (0,2); plates (-2,0)
    phioarr = [.01]
    #reqarr = [1,2,3,4,5,6,7,8,9,10,20,30,40,50,60,70,80,90,100,200,300,400,500,600,700,800,900,1000]
    reqarr = [10] 
    nclusters = 300         #changes how many aggregates per aspect ratio to consider
    ncrystals = 2
    rand_orient = False      #randomly orient the seed crystal and new crystal: uses first random orientation

    output = main()
    agg_as, agg_bs, agg_cs = compute()

AttributeError: 'ClusterCalculations' object has no attribute 'closest_points'

In [ ]:
# save results to file
results = {'agg_as': agg_as, 'agg_bs':agg_bs, 'agg_cs':agg_cs}
filename = '../instance_files/instance_iceice_flat_r10_phi20_run5'
filehandler = open(filename, 'wb')
pickle.dump(results, filehandler)
filehandler.close()
print('finished!')